# 1_load_thalamus_data

This notebook demonstrates how to use the custom thalamus_merfish_analysis 
module to load a standardized thalamus subset of the Allen Brain Cell (ABC) 
Atlas' whole mouse brain MERFISH dataset (https://portal.brain-map.org/atlases-and-data/bkp/abc-atlas).

It also includes descriptions of what data is included in this thalamus dataset. 

Additional information on the full ABC Atlas dataset can be found at: https://alleninstitute.github.io/abc_atlas_access/intro.html

In [1]:
from thalamus_merfish_analysis import abc_load as abc
get_ipython().run_line_magic('matplotlib', 'inline') 

## 1. Load thalamus dataset

You can load a thalamus subset of the ABC Atlas as either:

A) a pandas DataFrame, which includes just the cell metadata

B) an AnnData object, which includes:
- gene expression, stored in adata.X
- cell metadata, stored in adata.obs 
- gene metadata, stored in adata.var

The DataFrame is useful if you just want to explore mapped cell types and don't 
need the gene expression data. 

The DataFrame will load faster (~1 min vs. ~2.5 min) & take up less memory (~0.5 GB vs ~2 GB) than the full AnnData object.

### 1A. Load thalamus data as DataFrame

Load select cell metadata. See comment block below for details on what metadata
are included in the cell metadata DataFrame.

Additional information on these metadata, and other metadata available, can be 
found at: https://alleninstitute.github.io/abc_atlas_access/intro.html


In [2]:
'''
These metadata descriptions are compiled & modified from the "Allen Brain Cell 
Atlas - Data Access" companion Jupyter book, which can be found at: 
https://alleninstitute.github.io/abc_atlas_access/intro.html

For more details on the spatial coordinates, see: 
https://alleninstitute.github.io/abc_atlas_access/notebooks/merfish_ccf_registration_tutorial.html

For more information on the cell type taxonomy & definitions, see:
https://alleninstitute.github.io/abc_atlas_access/notebooks/cluster_annotation_tutorial.html

cell_label : str
    unique string used for ID of each cell; Index of the DataFrame
brain_section_label : str
    [brain specimen ID].[section number], e.g. "C57BL6J-638850.37".
    [brain specimen ID] is the same, 'C57BL6J-638850', for all cells in this  
    dataset. [section number] specifies the ordered index of each coronal 
    section, from 
neurotransmitter : str, {Glut, GABA, None, Dopa, Glut-GABA}
    neurotransmitter type of the cell; assigned based on average expression of  
    both neurotransmitter transporter genes and key neurotransmitter 
    synthesizing enzyme genes
class : str
    top level of cell type definition, primarily determined by broad brain 
    region and neurotransmitter type. Classes group together related subclasses 
    & all cells within a subclass belong to the same class.
    Class names are constructed as "[class ID] [brain region abbrv] 
    [neurotransmitter abbrv]", e.g. "20 MB GABA". 
subclass:
    a coarse level of cell type definition. Subclass groups together related 
    supertypes & all cells within a supertype belong to the same subclass
    Class names are constructed as "[subclass ID] [select marker genes] 
    [neurotransmitter abbrv]", e.g. "197 SNr Six3 Gaba".
supertype:
    second finest level of cell type definition; groups together similar 
    clusters & all cells within a cluster belong to the same supertype.
    Supertype names are constructed as "[supertype ID] [parent subclass label]_
    [supertype # within parent subclass]", e.g. "0806 SNr Six3 Gaba_1" and 
    "0806 SNr Six3 Gaba_2"
cluster : str
    finest level of cell type definition; cells within a cluster share similar 
    characteristics and belong to the same supertype.
    Cluster names are constructed as "[cluster ID] [parent supertype label]",
    e.g. "3464 SNr Six3 Gaba_1"
cluster_alias : int?
    unique 4-digit integer to identify the cluster to which the cell was mapped
average_correlation_score: float in range [0,1]
    correlation score specifying how "well" each cell mapped to it's assigned cluster
x_section, y_section, z_section : float
    original experiment coordinate space for MERFISH dataset. x & y specify the
    coronal plane (M-L & D-V, respectively). z specifies the section in A-P, and 
    all cells from the same experimental section have the same z_section
x_reconstructed, y_reconstructed, z_reconstructed : float
    point-to-point mapping between the original MERFISH coordinate space and the
    CCF space to achieve a finer level match to the target CCF section. x & y 
    specify the coronal plane, z specifies the sagittal plane and can vary for 
    cells from the same MERFISH z_section
z_ccf, y_ccf, x_ccf : float
    3D global affine mapping that aligns CCF into the MERFISH space. z & y 
    specify the coronal plane, medial-lateral & dorsal-ventral, respectively. 
    x specifies anterior-posterior.
parcellation_index :
    unique integer identifying each parcellation_substructure; used as the pixel
    value in the annotation volume
parcellation_division, parcellation_structure, parcellation_substructure : str
    human readable Allen Reference Atlas (ARA) parcellation levels to which the 
    cell belongs; division is the highest level, substructure is the lowest
left_hemisphere : bool
    True if cell is in the left hemisphere, False if in the right
'''

In [3]:
# Load cell metadata DataFrame
obs = abc.load_standard_thalamus(data_structure='obs')

In [4]:
# Display some info about the loaded DataFrame

# number of cells in this thalamus dataset
print(f'n_cells = {obs.shape[0]}')

# all metadata field names
display(obs.columns)

# first 11 metadata fields
display(obs.head(3).iloc[:, :11])
# last 11 metadata fields
display(obs.head(3).iloc[:, 11:])

### 1B. Load thalamus dataset as AnnData object

Includes:
- gene expression, stored in adata.X
- cell metadata, stored in adata.obs (identical to the DataFrame loaded in Part 1A)
- gene metadata, stored in adata.var
- the exact same set of cells as loaded into the DataFrame in Part 1A

In [5]:
'''
These metadata descriptions are compiled & modified from the "Allen Brain Cell 
Atlas - Data Access" companion Jupyter book, which can be found at: 
https://alleninstitute.github.io/abc_atlas_access/intro.html

adata_th.X : np.ndarray
    dense array of gene expression values for each cell in the dataset; standard 
    gene counts transform is log2cpm

adata_th.obs : pd.DataFrame
    cell metadata, identical to the DataFrame version loaded in Part 1A
    
adata_th.var : pd.DataFrame
    gene metadata, with the following fields:
    gene_symbol : str
        commonly used gene name. Both the Index of the DataFrame and a column
    transcript_identifier : str
        unique Ensembl transcript ID for each gene
'''

In [6]:
# Load thalamus AnnData object (includes gene expression + cell & gene metadata)
adata_th = abc.load_standard_thalamus(data_structure='adata')

In [7]:
# Display some info about the loaded AnnData object
display(adata_th)

display(adata_th.var.head(3))

display(adata_th.uns)

display(adata_th.X)